In [24]:
from sqlalchemy import create_engine
import names
import yaml
from random import randint
import datetime

In [2]:
with open("db.yml", "r") as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

In [3]:
user = cfg['mysql']['user']
pss = cfg['mysql']['passwd']
host = cfg['mysql']['host']
port = cfg['mysql']['port']
db = cfg['mysql']['db']

In [4]:
en = create_engine(f'mysql+pymysql://{user}:{pss}@{host}:{port}/{db}')

In [125]:
def fecha():
    año = str(randint(1980, 2002))
    mes = str(randint(1,12))
    if mes == "2":
        dia = str(randint(1,28))
    else:
        dia = str(randint(1,30))
    return año + '-' + mes + '-' + dia

In [126]:
def calcular_edad(f_nac):
    hoy = date.today()
    try:
        cumple = f_nac.replace(year=hoy.year)
    except ValueError:
        cumple = f_nac.replace(year=hoy.year, day=f_nac.day - 1) 
    if cumple > hoy:
        return hoy.year - f_nac.year - 1
    else:
        return hoy.year - f_nac.year

In [127]:
def direccion():
    calles = ['av. del Parque', 'Pirámide de la Luna', 'Retorno 48', 'Calzad de la Virgen', "Retorno 1"]
    return calles[randint(0,4)] + ' ' +'#' + str(randint(1,100))

In [128]:
def municipio():
    muns = ['Coyoacán', 'Alvaro Obregón', 'Azcapotzalco', 'Miguel Hidalgo', 'Tlalpan', 'Benito Juárez']
    return muns[randint(0, 5)]

In [129]:
def cod_postal():
    cps = ['04460', '14506', '90210', '15023', '04571', '23429', '12345']
    return cps[randint(0, 6)]

In [130]:
for i in range(100):
    nombre = names.get_first_name()
    apellidos = names.get_last_name() + ' ' + names.get_last_name()
    f_nac = fecha()
    dia, mes, anio = [int(v) for v in f_nac.split("-")]
    edad = calcular_edad(date(dia, mes, anio))
    direc = direccion()
    cp = cod_postal()
    mun = municipio()
    estado = "CDMX"
    tel = '55' + str(randint(10000000, 99999999))
    cad = f'INSERT INTO CLIENTES VALUES ({i+1},"{nombre}","{apellidos}",\'{f_nac}\',{edad},"{direc}","{cp}","{mun}","{estado}","{tel}")'
    conn = en.connect()
    conn.execute(cad)
    conn.close()

In [ ]:
CREATE TABLE PRESTAMOS(
	CONTRACTID INTEGER NOT NULL,
    CUSTOMERID INTEGER NOT NULL,
    FECHA_INI DATE NOT NULL,
    FECHA_FIN DATE NOT NULL,
    MONTO_TOTAL DECIMAL(8,2) NOT NULL,
    FREQ_PAGOS SMALLINT NOT NULL,
    MONTO_PAGOS DECIMAL(8,2)
	);


In [32]:
import dateutil.relativedelta

In [131]:
plazos = [1, 3, 6, 9, 12, 15, 18, 20, 24, 36, 48, 60, 72]
f = [15, 30]
for i in range(150):
    p = randint(0,11)
    f_1 = randint(0,1)
    cliente = randint(1,50)
    monto_inicial = randint(1000, 300000)
    fecha_i = str(date.today())
    fecha_aux = datetime.datetime.strptime(fecha_i, "%Y-%m-%d")
    fecha_f = str(fecha_aux + dateutil.relativedelta.relativedelta(months=plazos[p]))[:10]
    monto_total = monto_inicial
    freq = f[f_1]
    monto_pagos = monto_total/plazos[p]
    cad = f'INSERT INTO PRESTAMOS VALUES ({i+1},{cliente},\'{fecha_i}\', \'{fecha_f}\' ,{monto_total},{freq},{monto_pagos})'
    conn = en.connect()
    conn.execute(cad)
    conn.close()

In [132]:
import pandas as pd

In [133]:
df = pd.read_csv('..\\prestamos.csv', sep = ',', encoding = 'utf8')

In [134]:
df.head()

,CONTRACTID,CUSTOMERID,FECHA_INI,FECHA_FIN,MONTO_TOTAL,FREQ_PAGOS,MONTO_PAGOS
0,1,21,13/09/2020,13/05/2022,123910,30,6195.50
1,2,43,13/09/2020,13/09/2024,30074,30,626.54
2,3,31,13/09/2020,13/09/2024,146714,30,3056.54
3,4,2,13/09/2020,13/10/2020,47609,15,47609.00
4,5,35,13/09/2020,13/05/2022,226255,30,11312.75


In [135]:
df.shape

(139, 7)

In [136]:
df.dtypes

CONTRACTID       int64
CUSTOMERID       int64
FECHA_INI       object
FECHA_FIN       object
MONTO_TOTAL      int64
FREQ_PAGOS       int64
MONTO_PAGOS    float64
dtype: object

In [138]:
for i , fila in df.iterrows():
    f = int(df['FREQ_PAGOS'].iloc[i])
    m = df['MONTO_PAGOS'].iloc[i]
    c = int(df['CONTRACTID'].iloc[i])
    fecha = df['FECHA_INI'].iloc[i]
    fecha_aux = fecha.split('/')
    dia = fecha_aux[0]
    mes = fecha_aux[1]
    anio = fecha_aux[2]
    fecha_aux_1 =  str(anio+ '-' +mes+ '-' +dia)
    fecha_aux = datetime.datetime.strptime(fecha_aux_1, "%Y-%m-%d")
    fecha_f = str(fecha_aux + dateutil.relativedelta.relativedelta(days=f))[:10]
    cad = f'INSERT INTO PAGOS VALUES({i+1},{c},\'{fecha_f}\',{m}, {m},\'{fecha_f}\')'
    conn = en.connect()
    conn.execute(cad)
    conn.close()